In [1]:
import re
from datasets import load_metric, load_from_disk, load_dataset, Features, Value, Sequence, DatasetDict, Dataset

def preprocess(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'#', ' ', text)
    text = re.sub(r"[^a-zA-Z0-9가-힣ㄱ-ㅎㅏ-ㅣぁ-ゔァ-ヴー々〆〤一-龥<>()\s\.\?!》《≪≫\'<>〈〉:‘’%,『』「」＜＞・\"-“”∧]", "", text)
    return text

def run_preprocess(data_dict):
    context = data_dict["context"]
    start_ids = data_dict["answers"]["answer_start"][0]
    
    before = data_dict["context"][:start_ids]
    after = data_dict["context"][start_ids:]
    
    process_before = preprocess(before)
    process_after = preprocess(after)
    process_data = process_before + process_after
    
    ids_move = len(before) - len(process_before)
    
    data_dict["context"] = process_data
    data_dict["answers"]["answer_start"][0] = start_ids - ids_move
    
    return data_dict

def check(data_list):
    for data in data_list:
        start_ids = data["answers"]["answer_start"][0]
        end_ids = start_ids + len(data["answers"]["text"][0])
        if data["answers"]["text"][0] != data["context"][start_ids : end_ids]:
            print("wrong")
            return
    print("good")

In [2]:
from datasets import load_metric, load_from_disk, load_dataset, Features, Value, Sequence, DatasetDict, Dataset

train_data = load_from_disk("/opt/ml/input/data/data/train_dataset")["train"]
val_data = load_from_disk("/opt/ml/input/data/data/train_dataset")["validation"]
test_data = load_from_disk("/opt/ml/input/data/data/test_dataset")["validation"]

In [3]:
new_train_data = []
for data in train_data:
    new_data = run_preprocess(data)
    new_train_data.append(new_data)
    
check(new_train_data)

good


In [4]:
new_val_data = []
for data in val_data:
    new_data = run_preprocess(data)
    new_val_data.append(new_data)
    
check(new_val_data)

good


In [5]:
import pandas as pd

train_df = pd.DataFrame(new_train_data)
val_df = pd.DataFrame(new_val_data)

In [6]:
print(train_df.head())

     title                                            context  \
0    미국 상원  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...   
1   인사조직관리  '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...   
2      강희제  강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...   
3   금동삼존불감  불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...   
4  계사명 사리구  동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...   

                                  question            id  \
0         대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?  mrc-1-000067   
1                   현대적 인사조직관리의 시발점이 된 책은?  mrc-0-004397   
2           강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?  mrc-1-000362   
3  11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?  mrc-0-001510   
4               명문이 적힌 유물을 구성하는 그릇의 총 개수는?  mrc-0-000823   

                                         answers  document_id  \
0        {'answer_start': [229], 'text': ['하원']}        18293   
1  {'answer_start': [212], 'text': ['《경영의 실제》']}        51638   
2        {'answer_start': [503], 'text': ['백성']}     

In [7]:
print(val_df.head())

   title                                            context  \
0    전효숙  순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...   
1  스토우빌선  요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...   
2     촐라  촐라 정부 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 권력...   
3  백조 기사  프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...   
4  탈관료제화  탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...   

                               question            id  \
0        처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?  mrc-0-003264   
1  스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?  mrc-0-004762   
2       촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?  mrc-1-001810   
3              로타이르가 백조를 구하기 위해 사용한 것은?  mrc-1-000219   
4    의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?  mrc-1-000285   

                                       answers  document_id  __index_level_0__  
0    {'answer_start': [281], 'text': ['한보철강']}         9027               2146  
1   {'answer_start': [146], 'text': ['1871년']}        51765               3106  
2     {'answer_start': [509], 'text

In [8]:
test_f = Features({'context': Value(dtype='string', id=None),
                'id': Value(dtype='string', id=None),
                'question': Value(dtype='string', id=None)})

train_f = Features({'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
                    'context': Value(dtype='string', id=None),
                    'id': Value(dtype='string', id=None),
                    'question': Value(dtype='string', id=None)})

In [9]:
train_datasets = DatasetDict({'train': Dataset.from_pandas(train_df, features=train_f), 'validation': Dataset.from_pandas(val_df, features=train_f)})

In [10]:
print(train_datasets)

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question'],
        num_rows: 240
    })
})


In [11]:
import pickle
def save_pickle(save_path, data_set):
    file = open(save_path, "wb")
    pickle.dump(data_set, file)
    file.close()

def get_pickle(pickle_path):
    f = open(pickle_path, "rb")
    dataset = pickle.load(f)
    f.close()
    return dataset

save_pickle("/opt/ml/lastcode/dataset/preprocess_train.pkl", train_datasets)

In [12]:
preprocess_dataset= get_pickle("/opt/ml/lastcode/dataset/preprocess_train.pkl")
print(preprocess_dataset)

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question'],
        num_rows: 240
    })
})


In [13]:
print(preprocess_dataset["validation"]["context"][200])

전용선 발착의 컨테이너 화물, 전용선 발착의 차급 화물을 취급하고 있다. 화물 열차는 하루 1회 왕복, 이나자와 역과의 전용화물열차가 운행되고 있다. 그 열차로 연결되었던 컨테이너 차나 유개차는 이나자와 역부터 먼저, 다른 열차로 주계되어서 일본 전국으로 수송되고 있다. 남문쪽의 측선의 나고야 방향부터, 역 남쪽에 있는 오지 제지 가스가이 공장으로 이어지는 전용선이 분기하고 있다. 전용선은 유개차나 컨테이너 차를 사용하는 종이 제품의 발송으로 사용하고 있다. 유개차의 발송처는 니자 화물 터미널 역의 IPC 니자. 전용선은 일요일 이외 매일 화차의 출입이 있어, 화차의 견인은 네피아의 로고를 붙였던 스위처가 행해지고 있다. 더욱이, 이 오지 제지 전용선은, 구 일본 육군 조병 공창 도리이마쓰 공장으로의 전용선을 운용했던 것이라고 되어 있다. 오지 제지 전용선부터 분기하는 아이치 전기 전용선도 있다. 대형 변압기 수송에 사용되고 있지만, 공장으로 직접 연결되지는 않고, 가스가이 시내에 있는 이 회사 공장부터 변압기를 반환하는 대형 차로 쌓이고 있다. 특수 수송때문에 빈번히 사용되는 일은 없다. 1945년경까지, 가스가이 시 니시야마 정 부근에 있던 나고야 육군 조병창 다카키 제조소 니시야마 분창(부지는 육상 자위대 가스가이 주둔지)으로 이어지는 전용선도 존재했다. 선로 터의 대부분이 도로로 전용되어, 일부에 철교가 남아져 있다.


In [14]:
preprocess_dataset["validation"]["answers"][0]

{'answer_start': [281], 'text': ['한보철강']}

In [15]:
import json
with open("/opt/ml/input/data/data/wikipedia_documents.json", "r") as f:
    wiki = json.load(f)
print(wiki["0"])

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.', 'corpus_source': '위키피디아', 'url': 'TODO', 'domain': None, 'title': '나라 목록', 'author': None, 'html': None, 'document_id': 0}


In [16]:
import re
from datasets import load_metric, load_from_disk, load_dataset, Features, Value, Sequence, DatasetDict, Dataset

def preprocess(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^a-zA-Z0-9가-힣ㄱ-ㅎㅏ-ㅣぁ-ゔァ-ヴー々〆〤一-龥<>()\s\.\?!》《≪≫\'<>〈〉:‘’%,『』「」＜＞・\"-“”∧]", "", text)
    return text

def run_preprocess(data_dict):
    context = data_dict["text"]
    process_data = preprocess(context)
    data_dict["text"] = process_data
    
    return data_dict

In [17]:
new_wiki = dict()
for ids in range(len(wiki)):
    new_wiki[str(ids)] = run_preprocess(wiki[str(ids)])

In [18]:
with open('/opt/ml/lastcode/dataset/preprocess_wiki.json', 'w', encoding='utf-8') as make_file:
    json.dump(new_wiki, make_file, indent="\t", ensure_ascii=False)

In [19]:
with open("/opt/ml/lastcode/dataset/preprocess_wiki.json", "r") as f:
    new_wiki = json.load(f)
print(new_wiki["0"])


{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다. # 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다. # 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다. 두 목록은 모두 가나다 순이다. 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.', 'corpus_source': '위키피디아', 'url': 'TODO', 'domain': None, 'title': '나라 목록', 'author': None, 'html': None, 'document_id': 0}


In [20]:
print(preprocess("#"))

#


In [30]:
print(new_wiki["432"]["text"])

게오르크 지멜(Georg Simmel, 1858년 3월 1일~1918년 9월 28일)은 독일 출신의 사회학자이다. 지멜은 사회학의 독특한 연구 분야를 그대로 그려 내고자 했다. 그는 만일 사회학이 모든 것을 포괄한다면, 그것은 연구 영역을 전혀 갖지 못하는 것이라는 점을 이유로 포괄적이고도 “종합적”인 과학으로서의 사회학이라는 허버트 스펜서와 오귀스트 콩트의 개념을 거부하였다. 따라서 하나의 포괄적인 자연과학이 있을 수 없고, 단지 개별적이고 전문적인 과학자들만이 있을 수 있듯이, 사회과학자에게도 각 분과는 그 고유 영역을 가져야만 한다. 과학이 되기 위해서 사회학은 과학적 방법으로 연구되어야만 하는 잘 규정된 연구 주제를 가져야 한다고 지멜은 제안하였다. 1858년 베를린에서 가톨릭으로로 개종한 부유한 유대인 상인의 아들로 태어나 1918년 슈트라스부르크에서 사망했다. 베를린 대학에서 역사와 철학을 공부했으며 <칸트의 물리적 단자론에서 본 물질의 본성>이라는 논문으로 박사 학위를 받았다. 왕성한 집필 활동으로 신문과 잡지 등에 200여 편의 글을 발표했고 <사회 분화론>, <돈의 철학>, <사회학의 근본 물음>을 포함하여 철학, 윤리학, 사회학 등 다양한 분야에 관한 저서들을 남겼다.


In [31]:
text_data = get_pickle("/opt/ml/lastcode/dataset/preprocess_train.pkl")

In [32]:
print(text_data)

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question'],
        num_rows: 240
    })
})


In [33]:
print(text_data["train"]["context"][0])

미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05
